In [0]:
%sql
with workspaces as (
     select workspace_id,
            workspace_name,
            workspace_url
       from system.access.workspaces_latest
), warehouse_scd as (
     select wh.*,
            w.workspace_name,
            w.workspace_url,
            wh.change_time as valid_from,
            lead(wh.change_time) over (
                partition by wh.workspace_id, wh.warehouse_id
                order by wh.change_time
            ) as valid_to,
            case
                when row_number() over (partition by wh.workspace_id, wh.warehouse_id order by wh.change_time) = 1 then 'add'
                when lead(wh.change_time) over (partition by wh.workspace_id, wh.warehouse_id order by wh.change_time) is null then 'delete'
                else 'update'
            end as change_op,
            case when delete_time is not null then 'deleted'
                 else 'active'
            end as is_deleted
       from system.compute.warehouses wh
       left join workspaces w
         on w.workspace_id = wh.workspace_id
      where wh.warehouse_id = :warehouse_id
), window_events as (
      select we.workspace_id, 
             we.warehouse_id,
             we.event_type,
             we.event_time,
             we.cluster_count,
             case when we.cluster_count = 0 then 'off'
                  when we.cluster_count > 0 then 'on'
             end as warehouse_state
        from users.steve_johansen.warehouse_scaling_events we
       where exists (select warehouse_id
                       from warehouse_scd wh
                       where we.workspace_id = wh.workspace_id)
         and event_time >= date_sub(current_timestamp(), :history_days)
), pre_window_event as (
      select pwe.workspace_id, 
             pwe.warehouse_id,
             pwe.event_type,
             pwe.event_time,
             pwe.cluster_count,
             case when pwe.cluster_count = 0 then 'off'
                  when pwe.cluster_count > 0 then 'on'
             end as warehouse_state
        from users.steve_johansen.warehouse_scaling_events pwe
       where event_time < date_sub(current_timestamp(), :history_days)
         and exists (select warehouse_id
                       from warehouse_scd wh
                       where pwe.workspace_id = wh.workspace_id)
     qualify row_number() over (partition by pwe.workspace_id, pwe.warehouse_id order by event_time desc) = 1
), all_events as (
      select *
        from window_events
       union all
      select * 
        from pre_window_event
), event_windows as (
      select workspace_id,
             warehouse_id,
             warehouse_state as window_state,
             event_time as event_window_start,
             lead(event_time, 1, current_timestamp()) over (partition by warehouse_id order by event_time) as event_window_end
        from all_events
), all_seconds as (
      select e.workspace_id,
             e.warehouse_id,
             e.window_state,
             cast(second_chunk as timestamp) as second_chunk
        from event_windows e
     lateral view explode(
                      sequence(cast(unix_timestamp(e.event_window_start) as bigint),
                               cast(unix_timestamp(e.event_window_end) - 1 as bigint), -- Subtract 1 to avoid off-by-one errors
                               1)
             ) as second_chunk
       where cast(unix_timestamp(e.event_window_start) as bigint) < cast(unix_timestamp(e.event_window_end) as bigint)
), raw_history AS (
      select workspace_id,
             warehouse_id, 
             query_second, 
             count(0) as queries_active
        from (select workspace_id,
                     warehouse_id, 
                     explode(sequence(unix_timestamp(date_trunc('SECOND', start_time)), 
                                      unix_timestamp(date_trunc('SECOND', end_time)), 
                                      1)
                     ) as query_second
                from users.steve_johansen.warehouse_query_history f
               where warehouse_id = :warehouse_id
                 and exists (select warehouse_id 
                              from warehouse_scd we 
                             where we.warehouse_id = f.warehouse_id
                               and we.workspace_id = f.workspace_id) -- Had scaling events (i.e., was on)
                 and start_time < current_timestamp()
                 and end_time > date_sub(current_timestamp(), :history_days)
                 and QueryRuntimeSeconds > 0
                 and statement_type is not null) q
       group by workspace_id,
                warehouse_id,
                query_second
), state_by_second AS (
      select s.workspace_id,
             s.warehouse_id,
             s.second_chunk,
             max(s.window_state) as warehouse_state,
             coalesce(max(rh.queries_active), 0) as query_load, -- queries_active will be null on this join because there were no queries for that warehouse active in the second
             case when coalesce(max(s.window_state), 'off') = 'off' then 'off'
                  when query_load > 0 then 'utilized'
                  when query_load = 0 and max(s.window_state) = 'on' then 'on_idle' -- queries_active will be null on this join because there were no queries for that warehouse active in the second
              end as utilization_flag
         from all_seconds s
         left join raw_history rh
           on s.warehouse_id = rh.warehouse_id
          and s.second_chunk::timestamp = rh.query_second::timestamp
        group by s.workspace_id,
                 s.warehouse_id,
                 s.second_chunk
)
select *
  from state_by_second

In [0]:
%sql
WITH q AS (
  WITH workspace as (
    SELECT
      account_id,
      workspace_id,
      workspace_name,
      workspace_url,
      status,
      concat(COALESCE(workspace_name, ''), ' (id: ', workspace_id, ')') AS workspace_full_name
    FROM
      system.access.workspaces_latest
  ),
  warehouse_snapshot AS (
    SELECT
      warehouse_id,
      CONCAT(w.warehouse_name, ' (id:', w.warehouse_id, ')') AS warehouse_name,
      w.workspace_id,
      w.account_id,
      change_time,
      delete_time,
      warehouse_channel,
      warehouse_type,
      warehouse_size,
      min_clusters,
      max_clusters,
      auto_stop_minutes,
      CASE
        WHEN delete_time IS NOT NULL THEN 'Deleted'
        ELSE 'Active'
      END AS is_deleted
    FROM
      system.compute.warehouses w
        INNER JOIN workspace
          ON workspace.workspace_id = w.workspace_id
    WHERE warehouse_id = :warehouse_id
    QUALIFY
      ROW_NUMBER() OVER (PARTITION BY w.workspace_id, warehouse_id ORDER BY change_time DESC) == 1
    LIMIT 1
  ),
  window_events AS (
    SELECT
      workspace_id,
      warehouse_id,
      event_type,
      event_time,
      cluster_count AS cluster_count,
      CASE
        WHEN cluster_count = 0 THEN 'OFF'
        WHEN cluster_count > 0 THEN 'ON'
      END AS warehouse_state
    FROM
      users.steve_johansen.warehouse_scaling_events we
    WHERE
      EXISTS (
        SELECT
          warehouse_id
        FROM
          warehouse_snapshot ss
        WHERE
          ss.warehouse_id = we.warehouse_id
      )
      AND event_time BETWEEN date_sub(current_timestamp(), :history_days) AND current_timestamp()
  ),
  pre_window_event AS (
    SELECT
      workspace_id,
      warehouse_id,
      event_type,
      event_time,
      cluster_count AS cluster_count,
      CASE
        WHEN cluster_count = 0 THEN 'OFF'
        WHEN cluster_count > 0 THEN 'ON'
      END AS warehouse_state
    FROM
      users.steve_johansen.warehouse_scaling_events pwe
    WHERE
      event_time < date_sub(current_timestamp(), :history_days)
      AND EXISTS (
        SELECT
          warehouse_id
        FROM
          warehouse_snapshot we
        WHERE
          we.warehouse_id = pwe.warehouse_id
      )
    QUALIFY
      ROW_NUMBER() OVER (PARTITION BY pwe.workspace_id, pwe.warehouse_id ORDER BY event_time DESC)
      = 1
  ),
  all_events AS (
    SELECT
      *
    FROM
      pre_window_event
    UNION ALL
    SELECT
      *
    FROM
      window_events
  ),
  event_windows AS (
    SELECT
      workspace_id,
      warehouse_id,
      warehouse_state AS window_state,
      event_time AS event_window_start,
      LEAD(event_time, 1, current_timestamp()) OVER (
          PARTITION BY warehouse_id
          ORDER BY event_time
        ) AS event_window_end
    FROM
      all_events
  ),
  all_seconds AS (
    SELECT
      e.workspace_id,
      e.warehouse_id,
      e.window_state,
      CAST(second_chunk AS TIMESTAMP) AS second_chunk
    FROM
      event_windows e
      LATERAL VIEW
        explode(
          sequence(
            CAST(UNIX_TIMESTAMP(e.event_window_start) AS BIGINT),
            CAST(UNIX_TIMESTAMP(e.event_window_end) - 1 AS BIGINT),
            1
          )
        ) AS second_chunk
    WHERE
      CAST(UNIX_TIMESTAMP(e.event_window_start) AS BIGINT) < CAST(
        UNIX_TIMESTAMP(e.event_window_end) AS BIGINT
      )
  ),
  raw_history AS (
    SELECT
      workspace_id,
      warehouse_id,
      query_second,
      COUNT(0) AS queries_active
    FROM
      (
        SELECT
          workspace_id,
          warehouse_id,
          explode(
            sequence(
              UNIX_TIMESTAMP(date_trunc('SECOND', start_time)),
              UNIX_TIMESTAMP(date_trunc('SECOND', end_time)),
              1
            )
          ) AS query_second
        FROM
          users.steve_johansen.warehouse_query_history f
        WHERE
          EXISTS (
            SELECT
              warehouse_id
            FROM
              warehouse_snapshot we
            WHERE
              we.warehouse_id = f.warehouse_id
              AND we.workspace_id = f.workspace_id
          )
          AND start_time < current_timestamp()
          AND end_time > date_sub(current_timestamp(), :history_days)
          AND QueryRuntimeSeconds > 0
          AND statement_type IS NOT NULL
      ) q
    GROUP BY
      workspace_id,
      warehouse_id,
      query_second
  ),
  state_by_second AS (
    SELECT
      s.workspace_id,
      s.warehouse_id,
      s.second_chunk,
      MAX(s.window_state) AS warehouse_state,
      COALESCE(MAX(rh.queries_active), 0) AS query_load,
      CASE
        WHEN COALESCE(MAX(s.window_state), 'OFF') = 'OFF' THEN 'OFF'
        WHEN query_load > 0 THEN 'UTILIZED'
        WHEN
          query_load = 0
          AND MAX(s.window_state) = 'ON'
        THEN
          'ON_IDLE'
      END AS utilization_flag
    FROM
      all_seconds s
        LEFT JOIN raw_history rh
          ON s.warehouse_id = rh.warehouse_id
          AND s.second_chunk::timestamp = rh.query_second::timestamp
    GROUP BY
      s.workspace_id,
      s.warehouse_id,
      s.second_chunk
  )
  SELECT
    *,
    case
      when :param_time_key = 'Year' then date_trunc('YEAR', second_chunk)
      when :param_time_key = 'Quarter' then date_trunc('QUARTER', second_chunk)
      when :param_time_key = 'Month' then date_trunc('MONTH', second_chunk)
      when :param_time_key = 'Week' then date_trunc('WEEK', second_chunk)
      when :param_time_key = 'Day' then date_trunc('DAY', second_chunk)
      when :param_time_key = 'Hour' then date_trunc('HOUR', second_chunk)
      else second_chunk
    end as time_key
  FROM
    state_by_second
  WHERE
    second_chunk BETWEEN date_sub(current_timestamp(), :history_days) AND current_timestamp()
)
SELECT
  `utilization_flag` `utilization_flag`,
  `time_key` `time_key`,
  COUNT(*) `count(*)`
FROM
  q
GROUP BY
  `time_key`,
  `utilization_flag`

In [0]:
%sql
with warehouses as (
    select *
      from (
        select *,
              row_number() over (partition by warehouse_id order by change_time desc) as rn
          from system.compute.warehouses
        )
     where rn = 1
),   query_counts as (
    select date_trunc('hour', h.end_time) as end_time_hour,
          h.compute.warehouse_id          as warehouse_id,
          w.warehouse_name                as warehouse_name,
          count(1)                        as query_count
      from system.query.history h
      left join warehouses w
        on h.compute.warehouse_id = w.warehouse_id
     where h.end_time >= current_timestamp() - interval 1 day
       and h.compute.warehouse_id is not null
       and h.workspace_id = '1444828305810485'
     group by h.compute.warehouse_id, w.warehouse_name, date_trunc('hour', h.end_time)
)
select end_time_hour as end_time,
       warehouse_name || ' (' || warehouse_id || ')' as warehouse,
       query_count
  from query_counts

In [0]:
%sql
select *
  from (
    select *,
           row_number() over (partition by warehouse_id order by change_time desc) as rn
      from system.compute.warehouses
  )
where rn = 1

In [0]:
%sql
select h.*
 from system.query.history h
where h.end_time >= current_timestamp() - interval 1 day
  and h.compute.warehouse_id is not null
  and h.workspace_id = '1444828305810485'
  and h.written_bytes > 1024 * 1024 * 100
order by end_time desc

In [0]:
%sql
with warehouses as (
    select *
      from (
        select *,
              row_number() over (partition by warehouse_id order by change_time desc) as rn
          from system.compute.warehouses
        )
     where rn = 1
),   written as (
    select date_trunc('hour', h.end_time) as end_time_hour,
          h.compute.warehouse_id          as warehouse_id,
          w.warehouse_name                as warehouse_name,
          sum(h.written_bytes)            as written_bytes
      from system.query.history h
      left join warehouses w                        
        on h.compute.warehouse_id = w.warehouse_id
     where h.end_time >= current_timestamp() - interval 1 day
       and h.compute.warehouse_id is not null
       and h.workspace_id = '1444828305810485'
     group by h.compute.warehouse_id, w.warehouse_name, date_trunc('hour', h.end_time)
)
select end_time_hour as end_time,
       warehouse_name || ' (' || warehouse_id || ')' as warehouse,
       written_bytes / 1024 / 1024 / 1024 / 1024 as written_gb
  from written
where written_bytes > 0

In [0]:
%sql
select *
  from system.compute.warehouse_events
where warehouse_id = '581ed75401284b97'
  and event_time >= current_timestamp() - interval 7 days
order by event_time desc

In [0]:
%sql
select 
  date_trunc('minute', event_time) - 
    make_interval(
      0, 0, 0, 0, 0, 
      extract(minute from event_time) % 10, 
      0
    ) as interval_start,
  max(cluster_count) as clusters_running
from system.compute.warehouse_events
where warehouse_id = '4b9b953939869799'
  and event_time >= current_timestamp() - interval 7 days
group by 
  date_trunc('minute', event_time) - 
    make_interval(
      0, 0, 0, 0, 0, 
      extract(minute from event_time) % 10, 
      0
    )
order by interval_start